In [1]:
import pandas as pd

C:\Users\marco\AppData\Local\Temp\ipykernel_26340\4080736814.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [1]:
import pandas as pd

# Carregar os dados
data = pd.read_csv('vgsales.csv')

# Ajustar os valores de vendas para a escala correta (multiplicar por 1.000.000)
sales_columns = ['NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales', 'Global_Sales']
for column in sales_columns:
    data[column] = data[column] * 1000000

# Criar as dimensões
dim_platform = data[['Platform']].drop_duplicates().reset_index(drop=True)
dim_platform['Platform_ID'] = dim_platform.index + 1

dim_genre = data[['Genre']].drop_duplicates().reset_index(drop=True)
dim_genre['Genre_ID'] = dim_genre.index + 1

dim_publisher = data[['Publisher']].drop_duplicates().reset_index(drop=True)
dim_publisher['Publisher_ID'] = dim_publisher.index + 1

dim_year = data[['Year']].drop_duplicates().reset_index(drop=True)
dim_year['Year_ID'] = dim_year.index + 1

locations = ['NA', 'EU', 'JP', 'Other', 'Global']
dim_location = pd.DataFrame(locations, columns=['Location'])
dim_location['Location_ID'] = dim_location.index + 1

# Juntar as dimensões ao DataFrame principal
data = pd.merge(data, dim_platform, on='Platform', how='left')
data = pd.merge(data, dim_genre, on='Genre', how='left')
data = pd.merge(data, dim_publisher, on='Publisher', how='left')
data = pd.merge(data, dim_year, on='Year', how='left')

# Criar a tabela de fatos
fact_table = data[['Year_ID', 'Platform_ID', 'Genre_ID', 'Publisher_ID', 'Name']].reset_index(drop=True)
fact_table['Game_ID'] = fact_table.index + 1 

# Processar as vendas por localização
game_location_sales = []

for index, row in data.iterrows():
    game_id = fact_table.loc[fact_table['Name'] == row['Name'], 'Game_ID'].values[0]
    sales = {
        'NA': row['NA_Sales'],
        'EU': row['EU_Sales'],
        'JP': row['JP_Sales'],
        'Other': row['Other_Sales'],
        'Global': row['Global_Sales'],
    }
    for loc, value in sales.items():
        game_location_sales.append({
            'Game_ID': game_id,
            'Location_ID': dim_location[dim_location['Location'] == loc]['Location_ID'].values[0],
            'Sales': value
        })

game_location_sales = pd.DataFrame(game_location_sales)

# Salvar os dados em um arquivo Excel
with pd.ExcelWriter('game_sales_data_with_locations.xlsx', engine='openpyxl') as writer:
    dim_platform.to_excel(writer, sheet_name='Dim_Platform', index=False)
    dim_genre.to_excel(writer, sheet_name='Dim_Genre', index=False)
    dim_publisher.to_excel(writer, sheet_name='Dim_Publisher', index=False)
    dim_year.to_excel(writer, sheet_name='Dim_Year', index=False)
    dim_location.to_excel(writer, sheet_name='Dim_Location', index=False)
    fact_table.to_excel(writer, sheet_name='Fact_Table', index=False)
    game_location_sales.to_excel(writer, sheet_name='Game_Location_Sales', index=False)

C:\Users\marco\AppData\Local\Temp\ipykernel_15628\1481342525.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
